<a href="https://colab.research.google.com/github/larasauser/master/blob/main/Whitt_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# === 0) Installer / importer libs et monter Drive ===
!pip install rasterio tqdm scikit-image scikit-learn piq --quiet

import os
from glob import glob
from datetime import datetime
from tqdm import tqdm
import numpy as np
import rasterio
from rasterio.warp import reproject, Resampling
from scipy.linalg import lstsq
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm

import torch
import piq
import matplotlib.pyplot as plt
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# === 1) Définir les chemins ===
base_dir = '/content/drive/My Drive/Whitt'
clear_dir = os.path.join(base_dir, 'NDVI_grancy_Landsat8')
masked_dir = os.path.join(base_dir, 'NDVI_grancy_Landsat8_hole')
mask_dir = os.path.join(base_dir, 'mask_g')

In [3]:
# === 2) Fonction utilitaire : extraction des dates ===
def extract_date_from_filename(fname):
    """
    Suppose un format de fichier : NDVI_YYYY-MM-DD.tif
    """
    name = os.path.basename(fname)
    date_str = name.split('_')[1].split('.')[0]
    return datetime.strptime(date_str, "%Y-%m-%d")

In [4]:
# === 3) Charger les fichiers Landsat ===
clear_files = sorted(glob(os.path.join(clear_dir, '*.tif')))
masked_files = sorted(glob(os.path.join(masked_dir, '*.tif')))

clear_dates = [extract_date_from_filename(f) for f in clear_files]
masked_dates = [extract_date_from_filename(f) for f in masked_files]

print(f"→ {len(clear_files)} images pleines chargées")
print(f"→ {len(masked_files)} images masquées chargées")

→ 174 images pleines chargées
→ 6 images masquées chargées


In [5]:
# === 4) Construire la timeline complète ===
all_dates = sorted(list(set(clear_dates + masked_dates)))
print(f"→ Timeline complète : {len(all_dates)} dates")

→ Timeline complète : 180 dates


In [6]:
# === 5) Chargement d'une image de référence ===
def load_ndvi_image(filepath):
    with rasterio.open(filepath) as src:
        return src.read(1)

template = load_ndvi_image(clear_files[0])
height, width = template.shape

In [9]:
# === 5bis) Trouver l'image Landsat la plus petite comme référence ===
def get_image_shape_and_meta(path):
    with rasterio.open(path) as src:
        return src.read(1).shape, src.meta.copy()

sizes = []
for f in clear_files + masked_files:
    shape, meta = get_image_shape_and_meta(f)
    sizes.append((shape[0]*shape[1], f, shape, meta))

# Trier par taille (nombre de pixels)
sizes.sort(key=lambda x: x[0])
_, ref_file, ref_shape, ref_meta = sizes[0]
H_ref, W_ref = ref_shape

print(f"📏 Référence choisie : {os.path.basename(ref_file)} ({H_ref}x{W_ref})")

📏 Référence choisie : NDVI_2013-06-05.tif (344x319)


In [10]:
# === 6) Fonction de resampling vers la référence ===
def resample_to_reference(src_arr, src_meta, ref_meta):
    """Reprojette et redimensionne une image NDVI vers la grille de référence."""
    dst_arr = np.full((ref_meta['height'], ref_meta['width']), np.nan, dtype=np.float32)
    reproject(
        source=src_arr,
        destination=dst_arr,
        src_transform=src_meta['transform'],
        src_crs=src_meta['crs'],
        dst_transform=ref_meta['transform'],
        dst_crs=ref_meta['crs'],
        resampling=Resampling.bilinear
    )
    return dst_arr

In [11]:
# === 7) Construire le stack harmonisé ===
ndvi_stack = np.full((len(all_dates), H_ref, W_ref), np.nan, dtype=np.float32)

def read_singleband_tif(path):
    with rasterio.open(path) as src:
        arr = src.read(1).astype(np.float32)
        meta = src.meta.copy()
    nd = meta.get('nodata', None)
    if nd is not None:
        arr[arr == nd] = np.nan
    return arr, meta

print("🔄 Alignement des images sur la grille de référence...")

for i, date in tqdm(enumerate(all_dates), total=len(all_dates)):
    if date in clear_dates:
        path = clear_files[clear_dates.index(date)]
    elif date in masked_dates:
        path = masked_files[masked_dates.index(date)]
    else:
        continue

    arr, meta = read_singleband_tif(path)

    # Harmonisation taille et projection
    if arr.shape != (H_ref, W_ref):
        arr = resample_to_reference(arr, meta, ref_meta)

    ndvi_stack[i] = arr

print("✅ NDVI stack harmonisé créé :", ndvi_stack.shape)

🔄 Alignement des images sur la grille de référence...


100%|██████████| 180/180 [00:03<00:00, 46.49it/s]

✅ NDVI stack harmonisé créé : (180, 344, 319)


In [12]:
print(ref_meta['crs'])
print(ref_meta['transform'])

EPSG:32632
| 30.00, 0.00, 299085.00|
| 0.00,-30.00, 5167395.00|
| 0.00, 0.00, 1.00|


In [20]:
# === 7bis) Conversion homogène des dates Landsat / masked en datetime.date ===
all_dates = sorted(list(set(clear_dates + masked_dates)))  # timeline complète
all_dates = [d.date() if isinstance(d, datetime) else d for d in all_dates]
clear_dates = [d.date() if isinstance(d, datetime) else d for d in clear_dates]
masked_dates = [d.date() if isinstance(d, datetime) else d for d in masked_dates]

In [21]:
# === 8) Construction du stack MODIS complet (pas de moyenne temporelle) ===
def extract_date_from_filename(fn):
    m = re.search(r'(\d{4}-\d{2}-\d{2})', os.path.basename(fn))
    if m:
        return datetime.strptime(m.group(1), '%Y-%m-%d').date()
    else:
        raise ValueError(f"Impossible d'extraire la date du fichier {fn}")

modis_files = sorted(glob('/content/drive/MyDrive/Whitt/NDVI_grancy_MODIS/*.tif'))
modis_dates = [extract_date_from_filename(f) for f in modis_files]
modis_map = {d: f for d, f in zip(modis_dates, modis_files)}

# Lecture + resampling
modis_stack_full = []
for d in tqdm(modis_dates, desc="Reading MODIS images"):
    arr, meta = read_singleband_tif(modis_map[d])
    if arr.shape != (H_ref, W_ref):
        arr = resample_to_reference(arr, meta, ref_meta)
    modis_stack_full.append(arr)
modis_stack_full = np.stack(modis_stack_full, axis=0)  # [num_modis_dates, H, W]
print("MODIS stack complet :", modis_stack_full.shape)

Reading MODIS images: 100%|██████████| 493/493 [00:19<00:00, 25.56it/s]

MODIS stack complet : (493, 344, 319)


In [23]:
# === 9) Fonctions EGF / Whittaker ===
def pearson_r(a, b):
    valid = ~np.isnan(a) & ~np.isnan(b)
    if valid.sum() < 2: return np.nan
    x, y = a[valid], b[valid]
    return np.corrcoef(x, y)[0, 1]

def compute_M_reference_pixel(i, j, landsat_ts, all_dates, modis_stack, modis_dates,
                              radius_pix=3, corr_thresh=0.3, window_days=32):
    T = len(all_dates)
    M_ref = np.full(T, np.nan)

    i0, i1 = max(0, i-radius_pix), min(modis_stack.shape[1], i+radius_pix+1)
    j0, j1 = max(0, j-radius_pix), min(modis_stack.shape[2], j+radius_pix+1)
    coords = [(ii,jj) for ii in range(i0,i1) for jj in range(j0,j1)]

    for t, landsat_date in enumerate(all_dates):
        corrs, series = [], []

        for ii, jj in coords:
            selected_idx = [k for k, md in enumerate(modis_dates)
                            if abs((md - landsat_date).days) <= window_days]
            if len(selected_idx) == 0:
                continue

            ms = modis_stack[selected_idx, ii, jj]
            ls = landsat_ts[t]
            if np.isnan(ls):
                continue

            r = pearson_r(ms, np.full_like(ms, ls))
            if not np.isnan(r) and r >= corr_thresh:
                corrs.append(r)
                series.append(np.nanmean(ms))

        if len(corrs) == 0:
            block = modis_stack[:, i0:i1, j0:j1]
            M_ref[t] = np.nanmean(block)
        else:
            corrs = np.array(corrs)
            weights = (corrs - corrs.min()) / (corrs.max()-corrs.min()+1e-9)
            weights /= weights.sum() + 1e-12
            M_ref[t] = np.sum(weights * np.array(series))

    return M_ref

def estimate_linear_transfer(M_ref, landsat_ts):
    valid = ~np.isnan(M_ref) & ~np.isnan(landsat_ts)
    if valid.sum() < 3: return None
    A_mat = np.vstack([M_ref[valid], np.ones(valid.sum())]).T
    y = landsat_ts[valid]
    sol, *_ = lstsq(A_mat, y)
    return sol[0], sol[1]

def whittaker_smoother(y, kappa=5.0):
    y = y.copy()
    mask = ~np.isnan(y)
    if mask.sum() == 0:
        return np.full_like(y, np.nan)
    y_filled = np.interp(np.arange(len(y)), np.arange(len(y))[mask], y[mask])
    n = len(y)
    D = np.zeros((n-2,n))
    for i in range(n-2):
        D[i, i:i+3] = [1, -2, 1]
    A = np.eye(n) + kappa*D.T@D
    z = np.linalg.solve(A, y_filled)
    return z

In [26]:
# === 10) Reconstruction EGF + Whittaker ===
WINDOW_METERS = 200
PIXEL_SIZE_M = 30
MODIS_WINDOW_DAYS = 32
KAPPA = 5
radius_pix = int(round((WINDOW_METERS / 2) / PIXEL_SIZE_M))

A = np.full((H_ref, W_ref), np.nan)
A0 = np.full((H_ref, W_ref), np.nan)
Mref_stack = np.full((len(all_dates), H_ref, W_ref), np.nan)

print("Calcul de M_ref et coefficients linéaires par pixel...")
for i in tqdm(range(H_ref)):
    for j in range(W_ref):
        landsat_ts = ndvi_stack[:, i, j]
        Mref = compute_M_reference_pixel(i, j, landsat_ts, all_dates,
                                         modis_stack_full, modis_dates,
                                         radius_pix=radius_pix,
                                         corr_thresh=0.3,
                                         window_days=MODIS_WINDOW_DAYS)
        Mref_stack[:, i, j] = Mref
        est = estimate_linear_transfer(Mref, landsat_ts)
        if est:
            A[i, j], A0[i, j] = est

# Reconstruction SLM
SLM = A[None, :, :] * Mref_stack + A0[None, :, :]

# Whittaker smoothing
print("Application Whittaker smoothing...")
smoothed_SLM = np.full_like(SLM, np.nan)
for i in tqdm(range(H_ref)):
    for j in range(W_ref):
        s = SLM[:, i, j]
        if np.all(np.isnan(s)): continue
        smoothed_SLM[:, i, j] = whittaker_smoother(s, kappa=KAPPA)

Calcul de M_ref et coefficients linéaires par pixel...


  0%|          | 0/344 [00:06<?, ?it/s]


KeyboardInterrupt: 